In [28]:
import os

if 'main_dir' in globals():
    os.chdir(main_dir)

import pandas as pd
from utils.load_config import load_exp_config, get_value_from_fields, get_df_historical_data
# import matplotlib

# %matplotlib inline

if os.getcwd().endswith("notebooks"):
    main_dir = os.getcwd()
    os.chdir("..")

In [29]:
file_path="/home/tcc/huyhoang/regelum-playground/regelum_data/outputs/2024-08-08/18-54-51/0/.callbacks/ValueCallback/Total_Objectives_in_iteration_00001.h5"
file_path="regelum_data/outputs/2024-08-08/21-18-31/0/.callbacks/HistoricalDataCallback/observations_actions_it_00001_ep_00001.h5"

file_path="/home/tcc/huyhoang/regelum-control/regelum_data/outputs/2024-08-12/18-28-16__ppo_3wrobot_kin/0/.callbacks/HistoricalDataCallback/observations_actions_it_00106_ep_00010.h5"

In [30]:
exp_path="/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-20/14-38-30/0"
chosen_name="observations_actions_it_00001"
df = get_df_historical_data(exp_path, chosen_name)

ref_exp_path="/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-20/14-45-59/0"
ref_chosen_name="observations_actions_it_00001"
ref_df = get_df_historical_data(ref_exp_path, ref_chosen_name)

## Get sampling time

In [31]:
def get_sampling_time(exp_path):
    config = load_exp_config(exp_path)

    sampling_time = config['scenario']['sampling_time']
    if "$" in sampling_time:
        correct_fields = sampling_time.strip("\{\}$").split(".")
        sampling_time = get_value_from_fields(config, correct_fields)

    return sampling_time

sampling_time = get_sampling_time(exp_path)
ref_sampling_time = get_sampling_time(ref_exp_path)

## Re-calculate objective

### Define objective function

In [32]:
from regelum.objective import RunningObjective
from regelum.model import ModelQuadLin
import numpy as np


objective_function = RunningObjective(ModelQuadLin(
    weights= [2, 10, 0, 0, 0],
    quad_matrix_type="diagonal",
    is_with_linear_terms=False,
))

observation = np.array([[1, 1, 0.01]])
actions = np.array([[1, 1]])
objective_function(observation, actions)

12.0

### Calculate Objective

In [33]:
df.loc[:, ["x_rob", "y_rob", "vartheta"]]
replacements = {
    "x_rob"   :     "x [m]", 
    "y_rob"   :    "y [m]",
    "vartheta":       "angle [rad]",
    "v"       : "velocity [m/s]",
    "omega"   :    "angular velocity [rad/s]",
}

new_columns = []
if "x [m]" not in df.columns.values:
    for col in df.columns.values:
        new_columns.append(col if col not in replacements else replacements[col])

df.columns = new_columns
df

,time,running_objective,current_value,episode_id,iteration_id,velocity [m/s],angular velocity [rad/s],x [m],y [m],angle [rad]
0,0.000000,102000.000000,10200.000000,1,1,0.048534,0.141204,-1.000000,-1.000000,0.000100
1,0.102297,101938.642611,20028.627300,1,1,0.085936,0.134733,-0.999695,-0.999905,-0.012354
2,0.204543,101221.443882,29438.599966,1,1,0.151771,0.120830,-0.996101,-0.999929,-0.002219
3,0.306383,99609.291621,38368.375468,1,1,0.092506,0.075487,-0.987976,-0.999885,0.011022
4,0.406574,97980.412795,46843.774485,1,1,0.074609,0.120221,-0.979701,-0.999758,0.019140
...,...,...,...,...,...,...,...,...,...,...
473,49.550836,3978.155569,176422.465451,1,1,-0.000627,-0.006858,-0.193734,-0.331120,1.439672
474,49.654525,3990.001288,176422.465459,1,1,0.013685,0.004465,-0.193973,-0.332952,1.441964
475,49.759354,3993.069816,176422.465467,1,1,-0.023515,-0.004269,-0.194035,-0.333429,1.442399
476,49.863719,3993.715791,176422.465475,1,1,0.052789,0.027240,-0.194048,-0.333529,1.443103


In [34]:
def cal_obj_df(row):
    observation = np.expand_dims(np.array(row.loc[["x [m]", "y [m]", "angle [rad]"]].values), axis=0)
    action = np.expand_dims(np.array(row.loc[["velocity [m/s]", "angular velocity [rad/s]"]].values), axis=0)
    
    return objective_function(observation, action)


df["objective_value"] = df.apply(cal_obj_df, axis=1)
df["accumulative_objective"] = df["objective_value"].apply(lambda x: x*sampling_time).cumsum()

ref_df["objective_value"] = ref_df.apply(cal_obj_df_new, axis=1)
ref_df["accumulative_objective"] = ref_df["objective_value"].apply(lambda x: x*ref_sampling_time).cumsum()

In [35]:
df

,time,running_objective,current_value,episode_id,iteration_id,velocity [m/s],angular velocity [rad/s],x [m],y [m],angle [rad],objective_value,accumulative_objective
0,0.000000,102000.000000,10200.000000,1,1,0.048534,0.141204,-1.000000,-1.000000,0.000100,12.000000,1.200000
1,0.102297,101938.642611,20028.627300,1,1,0.085936,0.134733,-0.999695,-0.999905,-0.012354,11.996887,2.399689
2,0.204543,101221.443882,29438.599966,1,1,0.151771,0.120830,-0.996101,-0.999929,-0.002219,11.983018,3.597991
3,0.306383,99609.291621,38368.375468,1,1,0.092506,0.075487,-0.987976,-0.999885,0.011022,11.949895,4.792980
4,0.406574,97980.412795,46843.774485,1,1,0.074609,0.120221,-0.979701,-0.999758,0.019140,11.914798,5.984460
...,...,...,...,...,...,...,...,...,...,...,...,...
473,49.550836,3978.155569,176422.465451,1,1,-0.000627,-0.006858,-0.193734,-0.331120,1.439672,1.171468,108.651765
474,49.654525,3990.001288,176422.465459,1,1,0.013685,0.004465,-0.193973,-0.332952,1.441964,1.183820,108.770147
475,49.759354,3993.069816,176422.465467,1,1,-0.023515,-0.004269,-0.194035,-0.333429,1.442399,1.187047,108.888852
476,49.863719,3993.715791,176422.465475,1,1,0.052789,0.027240,-0.194048,-0.333529,1.443103,1.187726,109.007625


## Plot objective

In [36]:
import matplotlib.pyplot as plt


fig, ax = plt.subplots(1, 2, figsize=(11, 5), sharey=True)
ax[0].set_title("PPO controller - final cost: {:0.2f}".format(df.accumulative_objective.max()))
df.plot(x="time", y="accumulative_objective", ax=ax[0])
df.plot(x="time", y="objective_value", ax=ax[0])

ax[1].set_title("Nominal controller - final cost: {:0.2f}".format(ref_df.accumulative_objective.max()))
ref_df.plot(x="time", y="accumulative_objective", ax=ax[1])
ref_df.plot(x="time", y="objective_value", ax=ax[1])

for _ax in ax:
    _ax.grid()

In [37]:

fig, ax = plt.subplots()
df.plot(x="x [m]", y="y [m]", ax=ax)
circle = plt.Circle((0, 0), 0.1, color="red")
ax.add_patch(circle)
ax.set_xlim(-2, 5)
ax.set_ylim(-2, 5)
ax.grid()